# Toy Neural Net

[Neural Network Simulator](https://www.mladdict.com/neural-network-simulator)

![mladdict](mladdict.png)

### To-Do
* make an intro explaining the purpose of this notebook
* 

## Theory

### Initialization

### Forward Pass

### Backpropogation Pass

### To-Do:
* Brush Up on the Chain Rule
* explain every formula using LaTex formulas

## Test Cases

I'm writing this test case to make about 5 test cases for the Neural Network, each with four iterations.
* one iteration consists of a forward pass and a backwards pass
* I will be using four iterations because the author uses 

### To-Do:
* brush up on Python Unit Testing

## Neural Network Exercise

### To-Do:
* create a skeleton for the user to implement their own code
    * include a class
    * maybe even include function headers
    * they will be responsible for writing the logic of each function (the underlying math)
* include guide to using test cases

## Neural Network (My Implementation)

### To-Do:
* implement it (do this before making the Neural Network Exercise)

## Data Collection

This is a little bit of extra code for creating the test datasets. Selenium is great for scraping this data because it can interact with the model. The data of each step and iteration will be stored in CSVs so the selenium script doesn't need to be run for each test case. 

* following [Selenium Python API Guide](https://selenium-python.readthedocs.io/index.html)
    * [Chapter 4](https://selenium-python.readthedocs.io/locating-elements.html) explains how to locate elements (I am using CSS selector)
* this repository uses [ChromeDriver](https://sites.google.com/a/chromium.org/chromedriver/downloads) for Linux (v77)



In [88]:
# Libraries

# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Python
import time

* [this article](https://markhneedham.com/blog/2015/07/15/python-unicodedecodeerror-ascii-codec-cant-decode-byte-0xe2-in-position-0-ordinal-not-in-range128/) explains all the weird unicode stuff (for the minus sign in the 'get_value' function

In [97]:
# Data Collection 

def test_vals_collect():
    
    def get_value(css_selector):
        
        num_string = driver.find_element_by_css_selector(css_selector).text.split('=')[-1]
        
        # this handles weird unicode negative values
        if not num_string.find(u'−'):
            return float(num_string.split(u'−')[1]) * -1
        
        return float(num_string)
    
    def one_step():
        driver.find_element_by_css_selector(
            '#app > div > div:nth-child(2) > div > div > div:nth-child(2) > div > div:nth-child(2) > div > div > div.player.text-center > button:nth-child(2)'
        ).click()

    # 0) Initialization
    
    selector_parent_path = '#app > div > div:nth-child(2) > div > div > div:nth-child(2) > div > div:nth-child(1) > div.col-md-10.col-sm-12.col-xs-12.vcenter > div > svg > svg:nth-child(2) >'
    
    w_2_11 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(1) > svg > text')
    w_2_21 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(4) > svg > text')
    w_2_12 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(2) > svg > text')
    w_2_22 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(5) > svg > text')
    w_2_13 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(3) > svg > text')
    w_2_23 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(6) > svg > text')

    w_3_11 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(1) > svg > text')
    w_3_21 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(2) > svg > text')
    w_3_31 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(3) > svg > text')

    # 1) Input Layer
    
    one_step()
    
    x_1 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(3) > div:nth-child(1) > div > div.panel-body > div:nth-child(1) > span > span > span.katex-html > span.base.displaystyle.textstyle.uncramped')
    x_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(3) > div:nth-child(1) > div > div.panel-body > div:nth-child(2) > span > span > span.katex-html > span.base.displaystyle.textstyle.uncramped')
    
    # 2) Hidden Layer
    
    one_step()
    
    a_1_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(3) > div:nth-child(2) > div > div.panel-body > div:nth-child(1) > span')
    a_2_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(3) > div:nth-child(2) > div > div.panel-body > div:nth-child(2) > span')
    a_3_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(3) > div:nth-child(2) > div > div.panel-body > div:nth-child(2) > span')
    
    # 3) Output Layer
    
    one_step()
    
    a_1_3 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(4) > div:nth-child(1) > div > div.panel-body > div > span > span > span.katex-html > span.base.displaystyle.textstyle.uncramped')

    # 4) Calculate the cost
    
    one_step()
    
    E = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(4) > div:nth-child(2) > div > div.panel-body > div > span')

    # 5) Error in the output layer
    
    one_step()
    
    d_1_3 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(6) > div:nth-child(1) > div > div.panel-body > div > span > span > span.katex-html > span.base.displaystyle.textstyle.uncramped')
    
    # 6) Error in the hidden layer
    
    one_step()
    
    d_1_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(6) > div:nth-child(2) > div > div.panel-body > div:nth-child(1) > span')
    d_2_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(6) > div:nth-child(2) > div > div.panel-body > div:nth-child(1) > span')
    d_2_3 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(6) > div:nth-child(2) > div > div.panel-body > div:nth-child(3) > span')
    
    # 7) Calculate the error with respect to weights between hidden and output layer
    
    one_step()
    
    dE_dw_11_3 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(1) > div > div.panel-body > div:nth-child(1) > span')
    dE_dw_21_3 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(1) > div > div.panel-body > div:nth-child(2) > span')
    dE_dw_31_3 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(1) > div > div.panel-body > div:nth-child(3) > span')
    
    # 8) Calculate the error with respect to weights between input and hidden layer
    
    one_step()
    
    dE_dw_11_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(2) > div > div.panel-body > div:nth-child(1) > span')
    dE_dw_12_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(2) > div > div.panel-body > div:nth-child(2) > span')
    dE_dw_13_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(2) > div > div.panel-body > div:nth-child(3) > span')
    dE_dw_21_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(2) > div > div.panel-body > div:nth-child(4) > span')
    dE_dw_22_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(2) > div > div.panel-body > div:nth-child(5) > span')
    dE_dw_23_2 = get_value('#app > div > div:nth-child(2) > div > div > div:nth-child(3) > div > div > div:nth-child(7) > div:nth-child(2) > div > div.panel-body > div:nth-child(6) > span')

    # 9) Update the weights between hidden and output layer
    
    one_step()
    
    back_w_3_11 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(1) > svg > text')
    back_w_3_21 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(2) > svg > text')
    back_w_3_31 = get_value(selector_parent_path + 'svg:nth-child(2) > svg:nth-child(3) > svg > text')

    # 10) Update the weights between input and hidden layer
    
    one_step()
    
    back_w_2_11 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(1) > svg > text')
    back_w_2_21 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(4) > svg > text')
    back_w_2_12 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(2) > svg > text')
    back_w_2_22 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(5) > svg > text')
    back_w_2_13 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(3) > svg > text')
    back_w_2_23 = get_value(selector_parent_path + 'svg:nth-child(1) > svg:nth-child(6) > svg > text')
    
    return 0 # object? (what would be easiest for writting to a csv)

#____________________________________________
# Running the Code

# Setup Driver / Launch Site
driver = webdriver.Chrome("./chromedriver")
driver.get("https://www.mladdict.com/neural-network-simulator")

test_vals_collect()

# End
driver.close()
